Criar um DataFrame manual que tenha o mesmo ID de venda repetido 3
vezes, mudando apenas a coluna data_atualizacao


In [0]:


dados_repetidos = [
  (101, "Camisa", 50.0, "2026-02-02 00:01:03"), 
  (101, "Camisa", 50.0, "2026-02-02 00:01:04"), 
  (101, "Camisa", 50.0, "2026-02-02 00:01:05")  
]

colunas = ["id_venda", "produto", "valor", "data_atualizacao"]

df_repeated = spark.createDataFrame(dados_repetidos, colunas)

display(df_repeated)


Window Function

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, desc, col 

# Criar uma janela particionada pelo id_venda e ordenada pela data_atualizacao decrescente.
window = Window.partitionBy("id_venda").orderBy(desc("data_atualizacao"))

# Criar uma coluna rn (row number) usando essa janela.
df_repeated = df_repeated.withColumn("rn", row_number().over(window))

# Criar uma janela particionada pelo id_venda e ordenada
window = Window.partitionBy("id_venda").orderBy(col("rn"))

# Filtrar apenas onde rn 1 (a linha mais recente).
df_repeated = df_repeated.where(col("rn") == 1)
display(df_repeated)

# Explicar no código (comentário) por que usamos row_number e não distinct .
# O distinct() não segue regra de qual excluir, ele remove linhas idênticas ou escolhe uma aleatória. O row_number() permite que você decida qual linha é a vencedora baseada em uma lógica como no exemplo a data mais recente de atualização do cadastro.